In [1]:
import xarray as xr

In [2]:
era5 = xr.open_zarr(
    'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr',
    chunks={'time': 48},
    consolidated=True,
    )
print("Model surface dataset size {} GiB".format(era5.nbytes/(1024**3)))
era5

/usr/local/anaconda3/envs/mlg_310/lib/python3.10/site-packages/xarray/core/dataset.py:273: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


Model surface dataset size 75.83941642194986 GiB


<xarray.Dataset> Size: 81GB
Dimensions:                                           (time: 92040,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 256B ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float64 512B ...
  * time                                              (time) datetime64[ns] 736kB ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 754MB dask.array<chunksize=(48, 64, 32), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(48, 13, 64, 32), meta=np.ndarray>